In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from AnimalShelterCRUD import AnimalShelter


# Data Manipulation / Model
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# Drop the _id column
df.drop(columns=['_id'], inplace=True)

# Breeds and counts
breed_counts = df['breed'].value_counts()

# Select the top 5 breeds
top_5_breeds = breed_counts.head(5)

# Dashboard Layout / View
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Div(
        style={'display': 'flex', 'align-items': 'center'},
        children=[
            html.Img(src=app.get_asset_url("Grazioso Salvare Logo.png"), style={'height':'5%', 'width':'5%'}),
            html.H1('SNHU CS-340 Dashboard - Dante Lee', style={'text-align': 'center', 'margin-left': '10px'})
        ]
    ),
    html.Hr(),
    html.Div(
        id='filter-container',
        className='row',
        style={'margin': '20px', 'border': '1px solid #ccc', 'padding': '10px'},
        children=[
            html.H3('Filter', style={'text-align': 'center'}),
            html.Div(
                className='col s12 m6 offset-m3',
                style={'display': 'flex', 'justify-content': 'center'},
                children=[
                    dcc.RadioItems(
                        id='filter-radio',
                        options=[
                            {'label': 'Water Rescue', 'value': 'Water Rescue'},
                            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                            {'label': 'Reset', 'value': 'Reset'}
                        ],
                        value='',
                        labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                    )
                ]
            )
        ]
    ),
    html.Div(
        id='charts',
        className='row',
        children=[
            
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '50%', 'float': 'left'},
            ),
            
            html.Div(
                id='chart-id',
                className='col s12 m6',
                style={'width': '50%', 'float': 'right'},
                children=[
                    dcc.Graph(
                        id='pie-chart',
                        figure=px.pie(values=top_5_breeds.values, names=top_5_breeds.index)
                    )
                ]
            ),
        ]
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        selected_rows=[0],  # Set the initial selected row to the first row

        # Selectable rows
        row_selectable="single",
    ),
    html.Br(),
    html.Hr()
])

#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Filter button callback
@app.callback(
    [Output('datatable-id', 'data'), Output('map-id', 'children'), Output('pie-chart', 'figure')],
    [Input('filter-radio', 'value'), Input('datatable-id', 'derived_virtual_data'), Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_data(filter_option, viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]
    
    filtered_data = shelter.filter_animals(filter_option)
    filtered_df = pd.DataFrame.from_records(filtered_data)
    filtered_df.drop(columns=['_id'], inplace=True)

    map_children = [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[filtered_df.iloc[row, 13], filtered_df.iloc[row, 14]],
                      children=[
                          dl.Tooltip(filtered_df.iloc[row, 4]),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(filtered_df.iloc[row, 9])
                          ])
                      ])
        ])
    ]

    pie_chart = px.pie(filtered_df['breed'].value_counts().nlargest(5), values='breed', names=filtered_df['breed'].value_counts().nlargest(5).index, title='Animal Breed')
    pie_chart.update_layout(title_x=0.5)

    return filtered_df.to_dict('records'), map_children, pie_chart



app.run_server(debug=True)

Dash app running on http://127.0.0.1:18980/
